In [1]:
# import os
# if '_cwd_set' not in locals(): locals()['_cwd_set'] = os.chdir(os.path.dirname(os.getcwd()))
import omnifig as fig
fig.initialize()
from src.jimports import *

In [1]:
import numpy as np

In [5]:
int(np.prod(()).item())

1

In [2]:
from src.op import MNIST

In [3]:
dataset = MNIST().load()

In [4]:
dataset

MNIST(image, label)

In [5]:
batch = dataset.batch(10)
batch['image'].shape

torch.Size([10, 28, 28])

In [10]:
batch

Batch(index, image, label, {size}, {num_iterations}, {drawn_batches}, {drawn_samples}, {epochs}, {epoch_seed}, {epoch_offset})

In [8]:
mnist = dataset._dataset

In [12]:
mnist.targets.shape

torch.Size([60000])

In [36]:
def format_value(val: float, width: int) -> str:
    import math

    if width < 1:
        raise ValueError("Width must be at least 1.")

    # Determine the sign and adjust width accordingly
    sign = '-' if val < 0 else ' '
    number_width = width - 1  # Account for the sign character

    abs_val = abs(val)

    # Try fixed-point notation
    integer_part = int(abs_val)
    integer_part_str = str(integer_part)
    integer_part_length = len(integer_part_str)

    if integer_part_length <= number_width:
        # Calculate available space for decimal places
        decimal_places = number_width - integer_part_length - (1 if integer_part_length < number_width else 0)
        format_str = f"{{:0{number_width}.{decimal_places}f}}"
        formatted_number = format_str.format(abs_val)
        if len(formatted_number) <= number_width:
            formatted = sign + formatted_number
            if len(formatted) == width:
                return formatted
    # Scientific notation
    if abs_val == 0:
        exponent = 0
    else:
        exponent = int(math.floor(math.log10(abs_val)))
    mantissa = abs_val / (10 ** exponent)

    # Available space for mantissa and exponent
    exponent_str = f"e{exponent}" if exponent >= 0 else f"e-{abs(exponent)}"
    exponent_length = len(exponent_str)
    mantissa_width = number_width - exponent_length
    if mantissa_width < 1:
        raise ValueError(f"Cannot format value {val} in width {width}")

    for decimal_places in range(mantissa_width, -1, -1):
        mantissa_rounded = round(mantissa, decimal_places)
        # Adjust mantissa and exponent if mantissa_rounded >= 10
        if mantissa_rounded >= 10:
            mantissa_rounded /= 10
            exponent += 1
            exponent_str = f"e{exponent}" if exponent >= 0 else f"e-{abs(exponent)}"
            exponent_length = len(exponent_str)
            mantissa_width = number_width - exponent_length
            if mantissa_width < 1:
                continue  # Not enough width, try next decimal_places

        if decimal_places > 0:
            mantissa_format = f"{{:.{decimal_places}f}}"
            mantissa_str = mantissa_format.format(mantissa_rounded)
        else:
            mantissa_rounded_int = int(round(mantissa_rounded))
            mantissa_str = f"{mantissa_rounded_int}"
            if mantissa_width >= len(mantissa_str) + 1:
                mantissa_str += '.'
        mantissa_str_length = len(mantissa_str)
        if mantissa_str_length <= mantissa_width:
            formatted = sign + mantissa_str + exponent_str
            if len(formatted) == width:
                return formatted
    raise ValueError(f"Cannot format value {val} in width {width}")


In [37]:
# Function to run a test case
def run_test(val, width, expected, test_case_number):
    try:
        formatted = format_value(val, width)
        if formatted != expected:
            print(f"Test case format_value({val},{width}) failed: got {formatted!r}, expected {expected!r}")
        else:
            pass  # Test passed
    except ValueError as e:
        if expected == 'ValueError':
            pass  # Expected ValueError
        else:
            print(f"Test case {test_case_number} failed: got ValueError {e}, expected {expected!r}")

# Test positive numbers
run_test(1.32, 6, " 1.320", 1)
run_test(10000, 6, " 10000", 2)
run_test(100003, 6, " 1.0e5", 3)
run_test(100003, 5, " 1.e5", 4)
run_test(100003, 4, " 1e5", 5)
run_test(100003, 3, 'ValueError', 6)
run_test(0.00003, 8, " 0.00003", 7)
run_test(0.00003, 7, " 3.0e-5", 8)
run_test(0.003, 6, " 0.003", 9)
run_test(123456, 7, " 123456", 10)

# Test negative numbers
run_test(-1.32, 6, "-1.320", 11)
run_test(-10000, 6, "-10000", 12)
run_test(-100003, 6, "-1.0e5", 13)
run_test(-100003, 5, "-1.e5", 14)
run_test(-100003, 4, "-1e5", 15)
run_test(-100003, 3, 'ValueError', 16)
run_test(-0.00003, 8, "-0.00003", 17)
run_test(-0.00003, 7, "-3.0e-5", 18)
run_test(-0.003, 6, "-0.003", 19)
run_test(-123456, 7, "-123456", 20)

# Test zero
run_test(0.0, 3, " 0.0", 21)
run_test(-0.0, 3, " 0.0", 22)
run_test(0.0, 5, " 0.000", 23)
run_test(-0.0, 5, " 0.000", 24)

# Test edge cases
run_test(100003, 3, 'ValueError', 25)
run_test(-100003, 3, 'ValueError', 26)
run_test(1234567, 7, 'ValueError', 27)
run_test(99999, 6, " 99999", 28)
run_test(-99999, 6, "-99999", 29)
run_test(99999.9, 7, " 99999.", 30)
run_test(0.0001, 6, " 0.000", 31)

# Test various widths
run_test(123.456, 7, " 123.456", 32)
run_test(123.456, 6, " 123.46", 33)
run_test(123.456, 5, " 123.5", 34)
run_test(123.456, 4, " 123", 35)
run_test(123.456, 3, "1.2e2", 36)
run_test(123.456, 2, 'ValueError', 37)

# format_value(6604e16, 6)
run_test(6604e16, 6, ' 7.e16', 38)

# Test length assertion
test_case_number = 38
for width in range(3, 10):
    for val in [0.12345, -0.12345, 12345.6789, -12345.6789]:
        try:
            formatted = format_value(val, width)
            if len(formatted) != width:
                print(f"Test case {test_case_number} failed: val={val}, width={width}, got length {len(formatted)}, expected {width}, formatted value: {formatted!r}")
            test_case_number += 1
        except ValueError:
            test_case_number += 1  # Expected for widths that are too small



Test case format_value(3e-05,7) failed: got ' 0.0000', expected ' 3.0e-5'
Test case format_value(-3e-05,7) failed: got '-0.0000', expected '-3.0e-5'
Test case format_value(0.0,3) failed: got ' 00', expected ' 0.0'
Test case format_value(-0.0,3) failed: got ' 00', expected ' 0.0'
Test case format_value(0.0,5) failed: got ' 0.00', expected ' 0.000'
Test case format_value(-0.0,5) failed: got ' 0.00', expected ' 0.000'
Test case format_value(1234567,7) failed: got ' 1.23e6', expected 'ValueError'
Test case format_value(99999.9,7) failed: got ' 100000', expected ' 99999.'
Test case format_value(123.456,7) failed: got ' 123.46', expected ' 123.456'
Test case format_value(123.456,6) failed: got ' 123.5', expected ' 123.46'
Test case format_value(123.456,5) failed: got ' 0123', expected ' 123.5'
Test case 36 failed: got ValueError Cannot format value 123.456 in width 3, expected '1.2e2'
Test case format_value(6.604e+19,6) failed: got ' 7.e19', expected ' 7.e16'


In [38]:
format_value(6604e16, 6)

' 7.e19'

In [43]:
format_value(np.pi, 6)

' 3.142'

In [53]:
from humanize import naturalsize

print(naturalsize(1234567890))

1.2 GB


'5G'

In [57]:
naturalsize?

Signature:
naturalsize(
    value: 'float | str',
    binary: 'bool' = False,
    gnu: 'bool' = False,
    format: 'str' = '%.1f',
) -> 'str'
Docstring:
Format a number of bytes like a human-readable filesize (e.g. 10 kB).

By default, decimal suffixes (kB, MB) are used.

Non-GNU modes are compatible with jinja2's `filesizeformat` filter.

Examples:
    ```pycon
    >>> naturalsize(3000000)
    '3.0 MB'
    >>> naturalsize(300, False, True)
    '300B'
    >>> naturalsize(3000, False, True)
    '2.9K'
    >>> naturalsize(3000, False, True, "%.3f")
    '2.930K'
    >>> naturalsize(3000, True)
    '2.9 KiB'
    >>> naturalsize(10**28)
    '10.0 RB'
    >>> naturalsize(10**34 * 3)
    '30000.0 QB'
    >>> naturalsize(-4096, True)
    '-4.0 KiB'

    ```

Args:
    value (int, float, str): Integer to convert.
    binary (bool): If `True`, uses binary suffixes (KiB, MiB) with base
        2<sup>10</sup> instead of 10<sup>3</sup>.
    gnu (bool): If `True`, the binary argument is ignored and 

In [2]:
# import mnist
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import datasets, transforms

In [3]:
mnist = datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.5,), (0.5,))
                          ]))
len(mnist)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 9.91M/9.91M [00:46<00:00, 212kB/s] 


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 28.9k/28.9k [00:00<00:00, 175kB/s]


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 1.65M/1.65M [00:29<00:00, 55.4kB/s]


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.13MB/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



60000

In [5]:
mnist.data.shape

torch.Size([60000, 28, 28])